# LEIA-ChipWhisperer Scripting Quickstart

**SUMMARY**: *In this lab, you will learn how to setup and connect to your ChipWhisperer hardware. We will also cover how to capture power traces, and how to communicate with target devices using Python scriping*

**LEARNING OUTCOMES:**
* Setting up LEIA Hardware
* Using the smartleia Python API to connect to your hardware
* Communication with the target using the ChipWhisperer SDK
* Capturing a power trace

## Prerequisites

Hold up - before continuing, ensure you have done the following:

* ☑ Run through the Jupyter introduction.

## Physical Setup

### Board pinout

![LEIA Solo Pinout](https://h2lab.org/images/devices/leia_pinout.png "LEIA Pinout")


### Testing points

![LEIA Solo test points TOP](https://h2lab.org/images/devices/leia_test_points_top.png)

![LEIA Solo test points BOT](https://h2lab.org/images/devices/leia_test_points_bot.png)

### Setting up measure Mode

In order to setup LEIA in the measure mode: 

 * Move the PRG1, PRG2, PRG3 (LEIA Solo < v1.4) and PRG4 to the LEIA position (1-2).
 * Remove the shunt bypass jumber if it is set.
 * Move the tearing jumper to the OFF position.
 * Setup the power source for the smartcard. We would advise an external "clean" power source for clean measurements. However, we are able to get proper traces with the USB-C power supply on the funcard. 
 

## Connecting to the ChipWhisperer

Now that your hardware is all setup, we can see how to connect to it. We can connect to the ChipWhisperer with the Python `import` and function:

In [ ]:
import chipwhisperer as cw
scope = cw.scope()

By default, ChipWhisperer will try to autodetect the type of device your are running (CWLite/CW1200 or CWNano), see the API documentation for manually specifying the scope type. If you have multiple ChipWhisperer devices connected, you will need to specify the serial number of the device you want to connect to:

```python
scope = cw.scope(sn='<some long string of numbers>')
```

For more information, see the API section of readthedocs.

Connecting to the target device is simple as well:

In [ ]:
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted

We will only be discussing the default target type, which is SimpleSerial. Other targets, like the CW305, will be covered in hardware specific demos. 

The typical default settings can be set using:

In [ ]:
scope.default_setup()

which from its [documentation](https://chipwhisperer.readthedocs.io/en/latest/api.html#chipwhisperer.capture.scopes.OpenADC.OpenADC.default_setup) you can see does the following for the CWLite/CW1200:

* Sets the scope gain to 45dB
* Sets the scope to capture 5000 samples
* Sets the scope offset to 0 (aka it will begin capturing as soon as it is triggered)
* Sets the scope trigger to rising edge
* Outputs a 7.37MHz clock to the target on HS2
* Clocks the scope ADC at 4\*7.37MHz. Note that this is *synchronous* to the target clock on HS2
* Assigns GPIO1 as serial RX
* Assigns GPIO2 as serial TX

## Video demonstration

Ajout video ici

## Communication with the Target using smartleia

Communication with targets, which is done through the `smartleia` python module.

### Import and connect to LEIA
We `import` the smartleia package

In [ ]:
import smartleia as sl

### Connect to the reader
Then we establish a connection to the LEIA reader by instanciating a `LEIA` class.

In [ ]:
reader = sl.LEIA()

### Open a connection to a smart card

Verify that the smart card is detected. This supposes of course that your smart card is inserted in the LEIA board ISO7816 connector!

In [ ]:
reader.is_card_inserted()
True

### Initialize a connection to a smart card
Now that the card is detected, we will configure it and detect its ATR (Answer To Reset) using the `configure_smartcard` method. Optional parameters allow to perform tuning on the PTS negociation phase as well as frequency of the communication (`protocol_to_use` chooses between T=0 and T=1, `freq_to_use` fixes the ISO7816 frequency, `negotiate_pts` allows PTS/PSS negotiation with the card, `negotiate_baudrate` allows ETU baudraute negotiation). Please refer to the ISO7816 standard as well as LEIA specific documentation for more details about these elements. The **default** parameters are conservative and should allow you to discuss with most smart cards, but the "power user" mode could be of use with some specific cards.

In [ ]:
reader.configure_smartcard(protocol_to_use=1,  # Use T=1
                           ETU_to_use=None,    # Let the reader determine the ETU to use
                           freq_to_use=None,   # Let the reader determine the freq to use
                           negotiate_pts=True, # Let the reader negotiate the PTS
                           negotiate_baudrate=True
)

The output of `configure_smartcard` on success should be the following, showing a successful discussion with the smart card. 

In [ ]:
reader.configure_smartcard(protocol_to_use=1)  # Use T=1
ATR = reader.get_ATR()
ATR
ATR(
    ts=0x3B,
    t0=0xF9,
    ta=[0x13, 0x00, 0xFE, 0x00],
    tb=[0x00, 0x00, 0x45, 0x00],
    tc=[0x00, 0x00, 0x00, 0x00],
    td=[0x81, 0x31, 0x00, 0x00],
    h=[0x4A, 0x43, 0x4F, 0x50,
       0x32, 0x34, 0x32, 0x52,
       0x33, 0x00, 0x00, 0x00,
       0x00, 0x00, 0x00, 0x00],### Initialize a connection to a card Forcing optionnal parameters

    t_mask=[0x05, 0x05, 0x01, 0x03],
    h_num=0x09,
    tck=0xA2,### Initialize a connection to a card Forcing optionnal parameters

    tck_present=0x01,
    D_i_curr=4,
    F_i_curr=372,
    f_max_curr=5000000,
    T_protocol_curr=1,
    ifsc=0
)

### We can pick parameters

In [ ]:
print(f"We are using protocol T={ATR.T_protocol_curr} and the frequence of the ISO7816 clock is {ATR.f_max_curr/1000} kHz !")
We are using protocol T=1 and the frequence of the ISO7816 clock is 5000.0 kHz !

### Sending APDU to the target
Now that everything is setup with the smart card, it is possible to send APDU and receive responses. An APDU can be forged with the `APDU` class constructor (taking classical CLA/INS/P1/P2 plus data), and then use the `send_APDU` method to send it, the output being a response of the `RESP` class (classical SW1 and SW2 plus data response).

In [ ]:
apdu_select = sl.APDU(cla=0x00, ins=0x01, p1=0x00, p2=0x00)
apdu_select
APDU(cla=0x0, ins=0x1, p1=0x0, p2=0x0, lc=0, le=0, send_le=1)
resp = reader.send_APDU(apdu_select)
resp
RESP(sw1=0x6D, sw2=0x00, le=0x0)
reader.close()

### Other things we can do …
#### Restart LEIA in DFU mode for flashing an update
It is possible to update the LEIA reader with a new firmware. In order to put LEIA in DFU mode, simply execute the `dfu()` method. After this, LEIA should be enumerated on the USB interface as a DFU class device, allowing to use DFU utilities such as the open source `dfu-util` CLI tool (that can be found [here](http://dfu-util.sourceforge.net/)).

In [ ]:
reader.dfu()

## Communication with the Target using the ChipWhisperer

Communication with targets is done through the ChipWisperer SDK.

First you need to import both `smartleia` and the `chipwhisperer` Python modules

In [ ]:
import chipwhisperer as cw
from chipwhisperer.capture import targets
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
from smartleia import APDU, TriggerPoints

### Configuring the ChipWhisperer scope
You can configure the ChipWhisperer scope with the following command.

In [ ]:
scope = cw.scope()
# setup scope parameters
scope.gain.gain = 45
scope.adc.samples = 10000
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.hs2 = "clkgen"

### Configuring the LEIA target
Now the specific LEIA target (as a ChipWhisperer target) can be configured using:

In [ ]:
target = cw.target(scope, targets.LeiaTarget)
target.init()
wait_for_card(target)
target.configure_smartcard()
target.select_applet()


### Waiting for the card to be inserted
Now wait for the smart card to be inserted with:

In [ ]:
wait_for_card(target)

### Reseting the target to a know state
At any time, it is possible to reset the target to a known initial state with:

In [ ]:
target.reset()

### Select the target AES applet
Once everything is setup, it is possible to select the target applet performing the AES with the `select_applet` method:

In [ ]:
 target.select_applet(applet=[0x45, 0x75, 0x74, 0x77, 0x74, 0x75, 0x36, 0x41, 0x70, 0x80])

### Setting the AES inputs
You can set the AES inputs (key and plaintext) with:

In [ ]:
KEY = "2b7e151628aed2a6abf7158809cf4f3c"
DATA = "6bc1bee22e409f96e93d7e117393172a"
CIPHER = "3ad77bb40d7a3660a89ecaf32466ef97"

set_inputs(target, KEY, DATA)

### Checking data on the target
You can check that the key and the plaintext has indeed been properly setup in the target using:

In [ ]:
target.checkEncryptionKey(key)
target.checkPlaintext(data)

### Define the trigger strategy
Now, in order to do some trace aquisition campaigns, it is necessary to setup triggers. The LEIA target allows to setup triggers in various points in time when sending an APDU and receiving a response using `set_trigger_strategy`, please refer to the `smartleia` documentation to have more information on this! As shown in the example below, the `TriggerPoints.TRIG_PRE_SEND_APDU` allows to launch a trigger juste before sending the APDU to the smart card, with a `delay` of 0.

In [ ]:
 #Point just before sending a simple APDU
target.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU],delay=0)
print(target.get_trigger_strategy(1))

## Capturing traces

Now that the target is programmed and that we know how to communicate with it, let us start recording some power traces! To capture a trace:

1. Arm the ChipWhisperer with `scope.arm()`. It will begin capturing as soon as it is triggered (which in our case is a rising edge on `gpio4`: we are in a specific case where the trigger comes from the smart card itself).
1. `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return code will tell you whether or not the scope timed out. It does not return the captured scope data.
1. You can read back the captured power trace with `scope.get_last_trace()`.


In [ ]:
# arm the scope
scope.arm()

# launch computation
target.go()

# wait for target to finish
while target.isDone() is False and timeout:
    timeout -= -1
    time.sleep(0.01)

try:
    ret = scope.capture()
    if ret:
        print("Timeout happened during acquisition")
except IOError as e:
    print(f"IOError: {e}")

cipher = target.readOutput()

# print the result value
cipherstr = ''.join('{:02x}'.format(x) for x in cipher)
print(cipherstr)

# print the expected value
expectedstr=''.join('{:02x}'.format(x) for x in target.getExpected()))
print(expectedstr)
   



## Showing the traces
You can plot the traces with:

In [ ]:
import matplotlib.pyplot as plt

trace = scope.get_last_trace()  

plt.plot(trace)
plt.show()


## Saving the traces
You can save the traces as `numpy` arrays using:

In [ ]:
import numpy as np
from datetime import datetime

now = datetime.now()
fmt_string = "{:02}{:02}_{}.npy"

traces = []     # captured traces array
datain = []     # testcase data input array
known_keys = [] # testcase key input array

traces.append(trace)
datain.append()
known_key.append

trace_array = np.asarray(traces)
datain_array = np.asarray(datain)
known_keys = np.asarray(keys)


trace_file_path = os.path.join(workdir, fmt_string.format(now.hour, now.minute, "traces"))
datain_file_path = os.path.join(workdir,fmt_string.format(now.hour, now.minute, "data"))
keys_file_path = os.path.join(workdir,fmt_string.format(now.hour, now.minute, "keys"))

np.save(trace_file_path, trace_array)
np.save(datain_file_path, datain_array)
np.save(keys_file_path, known_keys)


## Conclusion

And that's it! 
As a final step, we should disconnect from the hardware so it does not stay "in use" by this notebook.

In [ ]:
scope.dis()
target.dis()